In [1]:
from __future__ import annotations

from monty.serialization import loadfn

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client["matpes"]

In [19]:
# pbe = loadfn("2024_11_18_MatPES-20240214-r2SCAN-training-data.json.gz-training-data.json.gz")

In [21]:
def make_db(functional):
    raw = loadfn(f"2024_11_18_MatPES-20240214-{functional}-training-data.json.gz")
    data = []
    for k, v in raw.items():
        d = {"matpesid": k} | v
        comp = d["structure"].composition
        d["natoms"] = len(d["structure"])
        d["elements"] = list(comp.chemical_system_set)
        d["nelements"] = len(d["elements"])
        d["chemsys"] = comp.chemical_system
        d["formula"] = comp.reduced_formula
        d["composition"] = {el.symbol: amt for el, amt in comp.items()}
        d["structure"] = d["structure"].as_dict()
        data.append(d)
    collection = db[functional]
    collection.delete_many({})
    collection.insert_many(data)
    for k in ["natoms", "elements", "nelements", "chemsys", "formula", "matpesid"]:
        collection.create_index(k)

In [23]:
make_db("PBE")
#make_db("r2SCAN")